# Retrieve Information from Specific Data Corpus

In [0]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set up Azure OpenAI

In [0]:
!pip install -r /Workspace/Repos/jeanpierre.vanniekerk@truenorthgroup.co.za/document-analysis-using-gpt-3.ide/notebooks/requirements.txt

  Using cached azure_identity-1.6.0-py2.py3-none-any.whl (108 kB)
  Using cached streamlit-1.18.1-py2.py3-none-any.whl (9.6 MB)
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached matplotlib-3.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached plotly-5.12.0-py2.py3-none-any.whl (15.2 MB)
  Using cached scipy-1.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
  Using cached scikit_learn-1.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)
  Using cached tiktoken-0.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached llama_index-0.4.33.tar.gz (147 kB)
  Preparing metadata (setup.py) ... - done
  Using cached langchain-0.0.129-py3-none-any.whl (467 kB)
  Using cached faiss_cpu-1.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Using cached msal_extensions-0.3.1-py2.py3-

In [0]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

False

## Deploy a Language Model

In [0]:
# list models deployed with embeddings capability
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    if model["capabilities"]["embeddings"] != True:
        continue
    
    deployment_id = deployment["id"]
    break

# if not model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')
    model = "text-similarity-davinci-001"

    # Now let's create the deployment
    print(f'Creating a new deployment with model: {model}')
    result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {model} with deployment_id {deployment_id}')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

Found a succeeded deployment that supports embeddings with id: text-embedding-ada-002.


## Create Embeddings

see [01-get-embeddings.ipynb](./01-get-embeddings.ipynb) on how to get embeddings.

In this example, we will load embeddings from a file.

## Load Data

In [0]:
import pandas as pd
df_orig = spark.sql('select * from openai.document_analysis_embeddings').limit(100).toPandas()

In [0]:
import numpy as np

DEVELOPMENT = False  # Set to True for development using a subset of data

if DEVELOPMENT:
    # Sub-sample for development
    df = df_orig.sample(n=20, replace=False, random_state=9).copy()
else:
    df = df_orig.copy()

# drop rows with NaN
df.dropna(inplace=True)

# convert string to array
df["embedding"] = df['embedding'].apply(np.array)#.apply(eval)
df

,category,filename,title,content,embedding,n_tokens
0,business,001.txt,Ad sales boost Time Warner profit,""" Quarterly profits at US media giant TimeWarn...","[-0.01796060986816883, -0.01651429571211338, 0...",563
1,business,002.txt,Dollar gains on Greenspan speech,""" The dollar has hit its highest level against...","[-0.021637994796037674, -0.012706875801086426,...",457
2,business,003.txt,Yukos unit buyer faces loan claim,""" The owners of embattled Russian oil giant Yu...","[-0.01746140420436859, -0.03939249739050865, 0...",340
3,business,004.txt,High fuel prices hit BA's profits,""" British Airways has blamed high fuel prices ...","[-0.019902830943465233, -0.017122669145464897,...",558
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,"[-0.008347506634891033, -0.008401579223573208,...",385
...,...,...,...,...,...,...
95,business,096.txt,Burren awarded Egyptian contracts,""" British energy firm Burren Energy has been a...","[-0.005239638965576887, -0.023387936875224113,...",180
96,business,097.txt,'Strong dollar' call halts slide,""" The US dollar's slide against the euro and y...","[-0.04523226246237755, -0.03272397071123123, 0...",558
97,business,098.txt,IMF 'cuts' German growth estimate,""" The International Monetary Fund is to cut it...","[-0.01721673458814621, -0.019378477707505226, ...",285
98,business,099.txt,"GM, Ford cut output as sales fall",""" US car firms General Motors (GM) and Ford ha...","[-0.026180488988757133, -0.013377368450164795,...",551


## Count tokens

In [0]:
import tiktoken
encoding = tiktoken.get_encoding('gpt2')

df['token_count'] = ''

for idx, title, content in zip(df.index.values, df['title'].loc[df.index.values], df['content'].loc[df.index.values]):
    df['token_count'].loc[idx] = len(encoding.encode(content))

df

<command-1550531866917500>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['token_count'].loc[idx] = len(encoding.encode(content))


,category,filename,title,content,embedding,n_tokens,token_count
0,business,001.txt,Ad sales boost Time Warner profit,""" Quarterly profits at US media giant TimeWarn...","[-0.01796060986816883, -0.01651429571211338, 0...",563,555
1,business,002.txt,Dollar gains on Greenspan speech,""" The dollar has hit its highest level against...","[-0.021637994796037674, -0.012706875801086426,...",457,459
2,business,003.txt,Yukos unit buyer faces loan claim,""" The owners of embattled Russian oil giant Yu...","[-0.01746140420436859, -0.03939249739050865, 0...",340,341
3,business,004.txt,High fuel prices hit BA's profits,""" British Airways has blamed high fuel prices ...","[-0.019902830943465233, -0.017122669145464897,...",558,546
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,"[-0.008347506634891033, -0.008401579223573208,...",385,382
...,...,...,...,...,...,...,...
95,business,096.txt,Burren awarded Egyptian contracts,""" British energy firm Burren Energy has been a...","[-0.005239638965576887, -0.023387936875224113,...",180,173
96,business,097.txt,'Strong dollar' call halts slide,""" The US dollar's slide against the euro and y...","[-0.04523226246237755, -0.03272397071123123, 0...",558,552
97,business,098.txt,IMF 'cuts' German growth estimate,""" The International Monetary Fund is to cut it...","[-0.01721673458814621, -0.019378477707505226, ...",285,278
98,business,099.txt,"GM, Ford cut output as sales fall",""" US car firms General Motors (GM) and Ford ha...","[-0.026180488988757133, -0.013377368450164795,...",551,532


## Find documents with similar embeddings to the embeddings of the question

In [0]:
import numpy as np

def get_embedding(text, deployment_id=deployment_id):
    """ 
    Get embeddings for an input text. 
    """
    result = openai.Embedding.create(
      deployment_id=deployment_id,
      input=text
    )
    result = np.array(result["data"][0]["embedding"])
    return result

def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    similarity = np.dot(x, y)
    return similarity 

def order_document_sections_by_query_similarity(query, contexts):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

## Construct prompt
Add relevant document sections to the query prompt.

In [0]:
MAX_SECTION_LEN = 1000
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [0]:
def construct_prompt(query: str, context_embeddings: pd.DataFrame, df: pd.DataFrame) -> str:
    """
    Append sections of document that are most similar to the query.
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(query, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section['token_count'] + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections, with indexes:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question truthfully using context, if unsure, say "I don't know."\n\nContext:\n"""
    prompt = header + "".join(chosen_sections) + "\n\n Q: " + query + "\n A:"
    
    return prompt

In [0]:
query = 'News about stock market.'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)

Selected 2 document sections, with indexes:
30
35
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " Irish shares have risen to a record high, with investors persuaded to buy into the market by low inflation and strong growth forecasts.  The ISEQ index of leading shares closed up 23 points to 6661.89 on Thursday, fuelled by strong growth in banking and financial stocks. A fall in the rate of inflation to 2.3% in January gave a fresh boost to shares which have advanced 4% this month. The economy is set for strong growth in 2005 while interest rates remain low.  Several of Ireland's biggest companies saw their market value hit recent highs on Thursday. Allied Irish Banks, Ireland's biggest company by capitalisation, touched a five year peak while Bank of Ireland shares rose to their highest level since August 2002.  Telecoms firm Eircom, which recently revealed that it would re-enter the Irish mobile phone market, hit a yearly high. Analysts said 

## Retrieve Information

In [0]:
def retrieve_information(prompt):
    try:
        # Request API
        response = openai.Completion.create(
            deployment_id= "text-davinci-003", # has to be deployment_id
            prompt=prompt,
            temperature=1,
            max_tokens=100,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=1
        )

        # response
        result = response['choices'][0]['text']; print(result)
    except Exception as err:
        print(idx)
        print(f"Unexpected {err=}, {type(err)=}")

    return 

In [0]:
query = 'News about stock market.'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
30
35
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " Irish shares have risen to a record high, with investors persuaded to buy into the market by low inflation and strong growth forecasts.  The ISEQ index of leading shares closed up 23 points to 6661.89 on Thursday, fuelled by strong growth in banking and financial stocks. A fall in the rate of inflation to 2.3% in January gave a fresh boost to shares which have advanced 4% this month. The economy is set for strong growth in 2005 while interest rates remain low.  Several of Ireland's biggest companies saw their market value hit recent highs on Thursday. Allied Irish Banks, Ireland's biggest company by capitalisation, touched a five year peak while Bank of Ireland shares rose to their highest level since August 2002.  Telecoms firm Eircom, which recently revealed that it would re-enter the Irish mobile phone market, hit a yearly high. Analysts said 

In [0]:
query = 'What is the state of the economy of the world?'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 1 document sections, with indexes:
29
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " The soaring cost of oil has hit global economic growth, although world's major economies should weather the storm of price rises, according to the OECD.  In its latest bi-annual report, the OECD cut its growth predictions for the world's main industrialised regions. US growth would reach 4.4% in 2004, but fall to 3.3% next year from a previous estimate of 3.7%, the OECD said. However, the Paris-based economics think tank said it believed the global economy could still regain momentum.  Forecasts for Japanese growth were also scaled back to 4.0% from 4.4% this year and 2.1% from 2.8% in 2005. But the outlook was worst for the 12-member eurozone bloc, with already sluggish growth forecasts slipping to 1.8% from 2.0% this year and 1.9% from 2.4% in 2005, the OECD said. Overall, the report forecast total growth of 3.6% in 2004 for the 30 member countrie

In [0]:
query = 'Summarise the state of the economy of the world?'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 1 document sections, with indexes:
43
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " The Japanese economy has officially gone back into recession for the fourth time in a decade.  Gross domestic product fell by 0.1% in the last three months of 2004. The fall reflects weak exports and a slowdown in consumer spending, and follows similar falls in GDP in the two previous quarters. The Tokyo stock market fell after the figures were announced, but rose again on a widespread perception that the economy will recover later this year. On Wednesday, the government revised growth figures from earlier in 2004 which, when taking into account performance in the most recent period, effectively tips Japan into recession.  A previous estimate of 0.1% growth between July and September was downgraded to a 0.3% decline. A recession is commonly defined as two consecutive quarters of negative growth, although the Japanese government takes other factors i

In [0]:
query = 'What is the most talked about technology?'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
91
10
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " Turkey's biggest private mobile firm could bail out of a $3bn ($1.6bn) deal to build a network in Iran after MPs there slashed its stake in the project.  Conservatives in parliament say Turkcell's stake in Irancell, the new network, should be cut from 70% to 49%. They have already given themselves a veto over all foreign investment deals, following allegations about Turkish firms' involvement in Israel. Turkcell now says it may give up on the deal altogether.  Iran currently has only one heavily congested mobile network, with long waiting lists for new subscribers. Turkcell signed a contract for the new network in September. The new operator planned to offer subscriptions for about $180, well below the existing firm's $500 price tag. But a parliamentary commission has now ruled that Turkcell's 70% controlling stake is too high. They say that Turk

In [0]:
query = 'List all the celebrities.'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
36
20
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " One of Japan's best-known businessmen was arrested on Thursday on charges of falsifying shareholder information and selling shares based on the false data.  Yoshiaki Tsutsumi was once ranked as the world's richest man and ran a business spanning hotels, railways, construction and a baseball team. His is the latest in a series of arrests of top executives in Japan over business scandals. He was taken away in a van outside one of his Prince hotels in Tokyo.  There was a time when Mr Tsutsumi seemed untouchable. Inheriting a large property business from his father in the 1960s, he became one of Japan's most powerful industrialists, with close connections to many of the country's leading politicians. He used his wealth and influence to bring the Winter Olympic Games to Nagano in 1998. But last year, he was forced to resign from all the posts he held 

In [0]:
query = 'What are the sports in the news?'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
13
39
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " The Daily and Sunday Telegraph newspapers are axing 90 journalist jobs - 17% of their editorial staff.  The Telegraph Group says the cuts are needed to fund an £150m investment in new printing facilities. Journalists at the firm met on Friday afternoon to discuss how to react to the surprise announcement. The cuts come against a background of fierce competition for readers and sluggish advertising revenues amid competition from online advertising. The National Union of Journalists has called on the management to recall the notice of redundancy by midday on Monday or face a strike ballot.  Pearson's Financial Times said last week it was offering voluntary redundancy to about 30 reporters.  The National Union of Journalists said it stood strongly behind the journalists and did not rule out a strike. ""Managers have torn up agreed procedures and kic

In [0]:
query = 'Tell me about all the football games.'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
39
61
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " UK sportswear firm Umbro has posted a 222% rise in annual profit after sales of replica England football kits were boosted by the Euro 2004 tournament.  Pre-tax profit for 2004 was £15.4m ($29.4m). Umbro, which recently lost sponsorship deals with Chelsea and Celtic, said on Thursday it had signed a new four-year agreement with Scottish club Rangers. It hopes 2005 sales will benefit from the launch of a new England replica shirt ahead of the 2006 World Cup. In January, Umbro announced its sponsorship agreement with Chelsea, which gave Umbro the lucrative right to make replica shirts, would end in 2006, five years earlier than expected. The firm, which is to receive a payment from Chelsea of £24.5m, said it is ""appraising a number of additional investment opportunities as a result of this compensation"" . Chief executive Peter McGuigan said the f

In [0]:
query = 'Who won in the swimming contest?'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
59
36
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " Another three US insurance executives have pleaded guilty to fraud charges stemming from an ongoing investigation into industry malpractice.  Two executives from American International Group (AIG) and one from Marsh & McLennan were the latest. The investigation by New York attorney general Eliot Spitzer has now obtained nine guilty pleas. The highest ranking executive pleading guilty on Tuesday was former Marsh senior vice president Joshua Bewlay.  He admitted one felony count of scheming to defraud and faces up to four years in prison. A Marsh spokeswoman said Mr Bewlay was no longer with the company. Mr Spitzer's investigation of the US insurance industry looked at whether companies rigged bids and fixed prices. Last month Marsh agreed to pay $850m (£415m) to settle a lawsuit filed by Mr Spitzer, but under the settlement it ""neither admits nor

In [0]:
query = 'Who won in the running competition?'
prompt = construct_prompt(query=query, context_embeddings=df['embedding'], df=df); print(prompt)
retrieve_information(prompt=prompt)

Selected 2 document sections, with indexes:
36
10
Answer the question truthfully using context, if unsure, say "I don't know."

Context:

* " One of Japan's best-known businessmen was arrested on Thursday on charges of falsifying shareholder information and selling shares based on the false data.  Yoshiaki Tsutsumi was once ranked as the world's richest man and ran a business spanning hotels, railways, construction and a baseball team. His is the latest in a series of arrests of top executives in Japan over business scandals. He was taken away in a van outside one of his Prince hotels in Tokyo.  There was a time when Mr Tsutsumi seemed untouchable. Inheriting a large property business from his father in the 1960s, he became one of Japan's most powerful industrialists, with close connections to many of the country's leading politicians. He used his wealth and influence to bring the Winter Olympic Games to Nagano in 1998. But last year, he was forced to resign from all the posts he held 